In [1]:
import requests
import json
import pickle
import time

In [2]:
baseurl="https://api.pushshift.io/reddit/search/submission?size=100&subreddit="
subreddit=["atheism","christianity"]
binary_scrapes=[]

In [3]:
word_count_threshold=20
min_posts_required=5000

In [4]:
def check(js,current_count,continue_flag,b_url,ss):
    valid_posts_retrieved=current_count
    data=js['data']
    
    for l in range(len(js['data'])):
        try:
            selftext=data[l]['selftext']
            len_selftext=len(selftext.split())

            if len_selftext >= word_count_threshold:
                valid_posts_retrieved+=1
                ss.append(selftext)
        except:
            continue
    
    if valid_posts_retrieved>min_posts_required:
        continue_flag=False
    else:
        continue_flag=True
    
    last_epoch=data[-1]['created_utc']
    url=f'{b_url}{last_epoch}'      
    
    print(f"Posts obtained: {valid_posts_retrieved}")
    
    return valid_posts_retrieved, continue_flag, url, saved_scrapes

In [ ]:
for sub in subreddit:
    b_url=baseurl+sub+"&before="
    url=b_url
    continue_flag=True
    current_count=0
    saved_scrapes=[]
    
    print("----------------")
    print(f"Running subreddit: {sub}")
    
    while continue_flag:
        resp=requests.get(url)
#         print(f"Fetching url: {url}")  #debug purpose only
        js=json.loads(resp.text)
        
        current_count, continue_flag , url, saved_scrapes= check(js,current_count,continue_flag,b_url,saved_scrapes)
        time.sleep(1)
    
    scrape_path=f"../data/scrapes_{sub}.pickle"
    with open(scrape_path,'wb') as handle:
        pickle.dump(saved_scrapes,handle)
    print(f"Dump file scrapes_{sub}.pickle saved")

----------------
Running subreddit: atheism
Posts obtained: 52
Posts obtained: 108
Posts obtained: 147
